<a href="https://colab.research.google.com/github/caselga/kaggle-gstore/blob/master/Kaggle_GStore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SETUP

In [0]:
"""
Given customer's behavior in Q2 and Q3, what is the expected consumption during the holiday season (Dec-Jan)

References:
https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
https://stackoverflow.com/questions/20209600/pandas-dataframe-remove-constant-column

"""

"\nGiven customer's behavior in Q2 and Q3, what is the expected consumption during the holiday season (Dec-Jan)\n\nReferences:\nhttps://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields\n"

In [0]:
!cat /proc/cpuinfo | grep 'model name'

model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz


In [0]:
!cat /proc/meminfo | grep 'Mem' #check RAM

MemTotal:       26751684 kB
MemFree:        23874780 kB
MemAvailable:   25726328 kB


In [0]:
from multiprocess import Pool
import multiprocess 
multiprocess.__version__

#time Google colab cells
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=8f93fa99da24716c614981eee00d8f764a04281f4daf55c2cc29b919a2fecaf0
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [0]:
import json
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format #suppress scientific notation
from pandas import json_normalize

from google.colab import drive
drive.mount('/content/gdrive')

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=1b1078781db24120f8f9b7382a1ef6575b8661d0e902f46ca0faeca2983fda62
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [10]:
#Get Kaggle API token manually and save to Google Drive
#Transfer token to Colab
!mkdir /root/.kaggle
!cp /content/gdrive/'My Drive'/Kaggle/kaggle.json /root/.kaggle #change to proper location of Kaggle API token
!chmod 600 /root/.kaggle/kaggle.json #owner can read/write but can't execute

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [0]:
def load_df(csv_path='train_v2.csv.zip', nrows=None):
    jsoncols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    skipcols = ['customDimensions','hits','socialEngagementType']
    
    df = pd.read_csv(csv_path,nrows=nrows, 
      usecols=lambda x: x not in skipcols,
      dtype={'channelGrouping': 'category',
        'date': np.uint32,
        'fullVisitorId': 'str',
        'visitId': np.uint32,
        'visitNumber': np.uint32,
        'visitStartTime': np.uint32},
      converters={column: json.loads for column in jsoncols}
    )
    dflist = [df]
    for column in jsoncols:
      columndf = json_normalize(df[column])
      columndf = columndf.loc[:, (columndf != columndf.iloc[0]).any()]
      columndf.columns = [f"{column}.{subcolumn}" for subcolumn in columndf.columns]
      dflist.append(columndf)

    df1 = pd.concat(dflist,axis=1).drop(columns=jsoncols)
    for col in df1.columns[df1.columns.str.contains('device|geoNetwork|source|medium')]: df1[col] = df1[col].astype('category') 
    for col in ['totals.hits','totals.pageviews','totals.sessionQualityDim','totals.timeOnSite','totals.transactions','totals.transactionRevenue']:
      df1[col] = df1[col].astype(float)

    #remove other columns
    df1.drop(columns=[col for col in df1.columns[df1.dtypes=='object'] if col!='fullVisitorId'],inplace=True)
    df1.info(null_counts=True)

    #delete vars
    del df,dflist

    return df1


time: 24.3 ms


# Data import

In [0]:
!kaggle competitions download -c ga-customer-revenue-prediction

train_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission_v2.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
dftrain = load_df(csv_path='train_v2.csv.zip', nrows=None)
dftest = load_df(csv_path='test_v2.csv.zip', nrows=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1708337 entries, 0 to 1708336
Data columns (total 25 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   channelGrouping            1708337 non-null  category
 1   date                       1708337 non-null  uint32  
 2   fullVisitorId              1708337 non-null  object  
 3   visitId                    1708337 non-null  uint32  
 4   visitNumber                1708337 non-null  uint32  
 5   visitStartTime             1708337 non-null  uint32  
 6   device.browser             1708337 non-null  category
 7   device.operatingSystem     1708337 non-null  category
 8   device.isMobile            1708337 non-null  category
 9   device.deviceCategory      1708337 non-null  category
 10  geoNetwork.continent       1708337 non-null  category
 11  geoNetwork.subContinent    1708337 non-null  category
 12  geoNetwork.country         1708337 non-null  category
 1

In [0]:
dftrain['Test'] = False;dftest['Test']=True
dfinal = pd.concat([dftrain,dftest],ignore_index=True)

dfinal['device.isMobile'] = np.where(dfinal['device.isMobile']==True,1,0) #cleanup

dfinal.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 26 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   channelGrouping            2109926 non-null  category
 1   date                       2109926 non-null  uint32  
 2   fullVisitorId              2109926 non-null  object  
 3   visitId                    2109926 non-null  uint32  
 4   visitNumber                2109926 non-null  uint32  
 5   visitStartTime             2109926 non-null  uint32  
 6   device.browser             2109926 non-null  object  
 7   device.operatingSystem     2109926 non-null  object  
 8   device.isMobile            2109926 non-null  int64   
 9   device.deviceCategory      2109926 non-null  category
 10  geoNetwork.continent       2109926 non-null  category
 11  geoNetwork.subContinent    2109926 non-null  category
 12  geoNetwork.country         2109926 non-null  object  
 1

In [0]:
savePath = "/content/gdrive/My Drive/Kaggle/gstore/gstoredf.feather"
dfinal.to_feather(savePath)

time: 3.29 s


# Exploratory Data Analysis

In [0]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
savePath = "/content/gdrive/My Drive/Kaggle/gstore/gstoredf.feather"
dfinal = pd.read_feather(savePath)
dfinal.info(null_counts=True)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2109926 entries, 0 to 2109925
Data columns (total 26 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   channelGrouping            2109926 non-null  category
 1   date                       2109926 non-null  uint32  
 2   fullVisitorId              2109926 non-null  object  
 3   visitId                    2109926 non-null  uint32  
 4   visitNumber                2109926 non-null  uint32  
 5   visitStartTime             2109926 non-null  uint32  
 6   device.browser             2109926 non-null  object  
 7   device.operatingSystem     2109926 non-null  object  
 8   device.isMobile            2109926 non-null  int64   
 9   device.deviceCategory      2109926 non-null  category
 10  geoNetwork.continent       2109926 non-null

In [0]:
dfinal.groupby('Test')['date'].agg([min,max])

,min,max
Test,,
False,20160801,20180430
True,20180501,20181015


In [0]:
for col in dfinal.columns[dfinal.columns.str.contains('geoNetwork')]: print(col);print(dfinal[col].value_counts())

geoNetwork.continent
Americas     1094335
Asia          482487
Europe        451907
Africa         42248
Oceania        36050
(not set)       2899
Name: geoNetwork.continent, dtype: int64
geoNetwork.subContinent
Northern America      962135
Southern Asia         153331
Western Europe        142747
Northern Europe       138770
Southeast Asia        137663
Eastern Asia          121096
South America          92125
Eastern Europe         86983
Southern Europe        83407
Western Asia           68139
Australasia            35694
Central America        35053
Northern Africa        19270
Western Africa          9312
Southern Africa         6970
Eastern Africa          5392
Caribbean               5022
(not set)               2899
Central Asia            2258
Middle Africa           1304
Melanesia                153
Micronesian Region       141
Polynesia                 62
Name: geoNetwork.subContinent, dtype: int64
geoNetwork.country
United States      898011
India              134217
United

# Prepare modeling data

In [0]:
"""
IV: customers aggregated transactions in May-Oct; DV: these customers' transaction in Dec to Jan

"""

In [0]:
##BUILD DV

#Dataframe for transactions in Dec'16-Jan'17 for customers who transacted in May-Oct '16
dv16 = dfinal[(dfinal.date.between(20161201,20170131)) &
       (dfinal['fullVisitorId'].isin(dfinal[dfinal.date.between(20160501,20161015)]['fullVisitorId'].tolist()))
].groupby('fullVisitorId')['totals.transactionRevenue'].sum().reset_index()
dv16['yearset'] = 2016

#Dataframe for transactions in Dec'17-Jan'18 for customers who transacted in May-Oct '17
dv17 = dfinal[(dfinal.date.between(20171201,20180131)) &
       (dfinal['fullVisitorId'].isin(dfinal[dfinal.date.between(20170501,20171015)]['fullVisitorId'].tolist()))
].groupby('fullVisitorId')['totals.transactionRevenue'].sum().reset_index()
dv17['yearset'] = 2017

dvdf = pd.concat([dv16,dv17],ignore_index=True)
dvdf['target'] = np.log(1+dvdf['totals.transactionRevenue'])
dvdf.drop(columns='totals.transactionRevenue',inplace=True)
dvdf.set_index(['yearset','fullVisitorId'],inplace=True)
dvdf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4179 entries, (2016, '000937838020298941') to (2017, '9996122476473160824')
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  4179 non-null   float64
dtypes: float64(1)
memory usage: 77.3+ KB
time: 1.09 s


In [0]:
dvdf.target.describe()

count   4179.0000
mean       1.0288
std        4.2190
min        0.0000
25%        0.0000
50%        0.0000
75%        0.0000
max       21.9489
Name: target, dtype: float64

time: 10.5 ms


In [0]:
"""
categorical: ['device.browser','device.operatingSystem','geoNetwork.subContinent','geoNetwork.country',
  'geoNetwork.region','geoNetwork.metro','geoNetwork.city','geoNetwork.networkDomain','trafficSource.source']
categorical with few categories: ['channelGrouping','device.deviceCategory','geoNetwork.continent','trafficSource.medium']

numerical: ['device.isMobile']
"""

In [0]:
##BUILD IV
from multiprocess import Pool

#customer transactions occuring on these time periods
dfinal['yearset'] = dfinal.date // 10000 #extract year portion from integer date
iv16df = dfinal[dfinal.date.between(20160501,20161015)].copy()
iv17df = dfinal[dfinal.date.between(20170501,20171015)].copy()
iv18df = dfinal[dfinal.date.between(20180501,20181015)].copy()

#CATEGORICAL
colList = ['channelGrouping','device.deviceCategory','geoNetwork.continent','trafficSource.medium'] + ['device.browser','device.operatingSystem','geoNetwork.subContinent','geoNetwork.country',\
  'geoNetwork.region','geoNetwork.metro','geoNetwork.city','geoNetwork.networkDomain','trafficSource.source']
#inner concat by columns (axis=1); then outer concat by rows
with Pool(processes=4) as p:
  iv_cat = pd.concat([pd.concat(p.starmap(
      lambda df,col: pd.DataFrame(df.groupby(['yearset','fullVisitorId'])[col].apply(lambda x: pd.Series.mode(x)[0])),
    [(df,col) for col in colList]), axis=1) for df in [iv16df,iv17df,iv18df]])

#NUMERICAL
numColList = dfinal.columns[dfinal.columns.str.contains('totals.')]
iv_num = pd.concat([pd.concat(
    [pd.DataFrame(df.groupby(['yearset','fullVisitorId']).agg({col:['sum','median']})) 
  for col in numColList],axis=1) 
for df in [iv16df,iv17df,iv18df]])
iv_num.columns = ["_".join(x) for x in iv_num.columns.ravel()]


time: 25min 44s


In [0]:
#print(iv_cat.shape);print(iv_num.shape);print(dvdf.shape)
modeldf = dvdf.merge(iv_cat,how='right',on=['yearset','fullVisitorId']).merge(iv_num,how='right',on=['yearset','fullVisitorId'])
modeldf.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 777293 entries, (2016, '000937838020298941') to (2018, '9999947552481876143')
Data columns (total 26 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   target                            4179 non-null    float64
 1   channelGrouping                   777293 non-null  object 
 2   device.deviceCategory             777293 non-null  object 
 3   geoNetwork.continent              777293 non-null  object 
 4   trafficSource.medium              777293 non-null  object 
 5   device.browser                    777293 non-null  object 
 6   device.operatingSystem            777293 non-null  object 
 7   geoNetwork.subContinent           777293 non-null  object 
 8   geoNetwork.country                777293 non-null  object 
 9   geoNetwork.region                 777293 non-null  object 
 10  geoNetwork.metro                  777293 non-null  object 
 11  ge

In [0]:
modeldf.reset_index(inplace=True)
modeldf.groupby('yearset').size()

yearset
2016    151127
2017    329636
2018    296530
dtype: int64

time: 207 ms


In [0]:
savePath = "/content/gdrive/My Drive/Kaggle/gstore/gstoremodeldf.feather"
modeldf.to_feather(savePath)

time: 1.3 s


# Modeling

In [0]:
!pip install pycaret

from pycaret.utils import enable_colab 
enable_colab()

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

savePath = "/content/gdrive/My Drive/Kaggle/gstore/gstoremodeldf.feather"
df = pd.read_feather(savePath)
df.info(null_counts=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 777293 entries, 0 to 777292
Data columns (total 28 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   yearset                           777293 non-null  int64  
 1   fullVisitorId                     777293 non-null  object 
 2   target                            4179 non-null    float64
 3   channelGrouping            

In [3]:
tr = df[df.yearset!=2018].copy()
tr['target'].fillna(0,inplace=True)
train_label = tr.target.values
train_data = tr.drop(['yearset','fullVisitorId','target'],axis=1).values

ts = df[df.yearset==2018].copy()
test_data = ts.drop(['yearset','fullVisitorId','target'],axis=1).values

print(train_data.shape);print(test_data.shape)

(480763, 25)
(296530, 25)


In [6]:
from catboost import Pool, CatBoostRegressor, cv

from sklearn import model_selection
test_size = 0.33
seed = 200
X_train, X_valid, Y_train, Y_valid = model_selection.train_test_split(train_data, train_label, test_size=test_size, random_state=seed)

# initialize Pool
train_pool = Pool(X_train, Y_train, cat_features=list(range(13)))
valid_pool = Pool(X_valid, Y_valid, cat_features=list(range(13)))

test_pool = Pool(test_data, cat_features=list(range(13))) 

# specify the training parameters
#,used_ram_limit='10gb'
iters=200
model = CatBoostRegressor(verbose=50,iterations=iters, depth=8, learning_rate=0.1, loss_function='RMSE',task_type = 'GPU',used_ram_limit='10gb')
#train the model
model.fit(train_pool,eval_set=valid_pool)

# evaluate model
print(model.score(X_train, Y_train))
print(model.score(X_valid, Y_valid))

# baseline model
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(Y_valid, np.zeros(Y_valid.shape[0])))

0:	learn: 0.3980427	test: 0.4160235	best: 0.4160235 (0)	total: 54ms	remaining: 10.7s
50:	learn: 0.3802079	test: 0.4140630	best: 0.4140578 (49)	total: 2.81s	remaining: 8.21s
100:	learn: 0.3716834	test: 0.4140210	best: 0.4137789 (85)	total: 5.56s	remaining: 5.45s
150:	learn: 0.3638306	test: 0.4142577	best: 0.4137789 (85)	total: 8.31s	remaining: 2.7s
199:	learn: 0.3566012	test: 0.4143026	best: 0.4137789 (85)	total: 11.1s	remaining: 0us
bestTest = 0.4137788989
bestIteration = 85
Shrink model to first 86 iterations.
0.0758162254823348
0.012744740094481344


0.4165508525509993

# Create Kaggle submission

In [13]:
import pandas as pd;
import numpy as np;

pred = pd.DataFrame({'fullVisitorId':ts.fullVisitorId,'PredictedLogRevenue':np.maximum(0,model.predict(test_pool))}) 

pred['fullVisitorId'] = pred.fullVisitorId.astype(str) #IMPORTANT to cast to string

print(pred.PredictedLogRevenue.describe())

count    296530.000000
mean          0.011168
std           0.042460
min           0.000000
25%           0.000000
50%           0.003279
75%           0.011435
max           5.328703
Name: PredictedLogRevenue, dtype: float64


In [11]:
pred.to_csv('gstore_submission.csv.gz',index=False,compression='gzip')
!kaggle competitions submit -c ga-customer-revenue-prediction -f gstore_submission.csv.gz -m 'Latest submission'

100% 4.31M/4.31M [00:08<00:00, 527kB/s]
Successfully submitted to Google Analytics Customer Revenue Prediction